In [8]:
import SimpleITK as sitk
import numpy as np
import tempfile
import os
import shutil

mr_path = '/workspace/data/1%/Task2/Abdomen/2ABB001/cbct_s2.nii.gz'
ct_path = '/workspace/data/1%/Task2/Abdomen/2ABB001/ct_s2.nii.gz'
mask_path = '/workspace/data/1%/Task2/Abdomen/2ABB001/mask_s2.nii.gz'

parameter_file = '/workspace/code/preprocessing/configs/param_def_cbct_AB.txt'

mr = sitk.ReadImage(mr_path)
ct = sitk.ReadImage(ct_path)
mask = sitk.ReadImage(mask_path)

In [9]:
temp_dir = tempfile.mkdtemp()
current_directory = os.getcwd()
os.chdir(temp_dir)

parameter = sitk.ReadParameterFile(parameter_file)

# Perform registration based on parameter file
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetParameterMap(parameter)
elastixImageFilter.SetFixedImage(mr)  # due to FOV differences CT first registered to MR an inverted in the end
elastixImageFilter.SetMovingImage(ct)
elastixImageFilter.SetFixedMask(mask)
elastixImageFilter.LogToConsoleOn()
elastixImageFilter.LogToFileOff()
elastixImageFilter.SetNumberOfThreads(16)
elastixImageFilter.Execute()


ELASTIX version: 5.0.1
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  16
Command line options from TransformBase:
-t0       unspecified, so no initial transform used
  The default value "3" is used instead.
  The default value "false" is used instead.

Reading images...
Reading images took 0 ms.

  The rigidity penalty term is evaluated on entire input transform domain.
Initialization of all components (before registration) took: 0 ms.
Preparation of the image pyramids took: 485 ms.

Resolution: 0
but the selected ImageSampler is not suited for that.
but the selected ImageSampler is not suited for that.
but the selected ImageSampler is not suited for that.
  The default value "true" is used instead.
  The default value "true" is used instead.
  The default value "true" is used instead.
  The default value "true" is used instead.
  The default value "true" is used instead.
  The def

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x7f471ee06ee0> >

In [10]:
img = elastixImageFilter.GetResultImage()
img[mask==0]=-1000
sitk.WriteImage(img, '/workspace/data/1%/Task2/Abdomen/2ABB001/ct_s2_reg.nii.gz')